In [7]:
#import libraries

import kaggle


In [29]:
#download dataset from kaggle

!kaggle datasets download ankitbansal06/retail-orders

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0


In [37]:
#extract file from zip file
import zipfile
zip_ref=zipfile.ZipFile("C:/Users/Lenovo/Documents/SQL + Python/retail-orders.zip")
zip_ref.extractall() #extract file to dir
zip_ref.close() #close file

In [19]:
#read data from the file and handle null values
import pandas as pd

# Use the 'python' engine to parse the file
df = pd.read_csv("orders.csv", na_values=['Not Available', 'unknown'])

# Display the first few rows of the DataFrame
df.head(20)
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [25]:
#rename column names ..make them lower case and replace spaces with underscore
df.rename(columns={'Order Id' : 'order_id', 'City' : 'city'})
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(' ', '_')


In [27]:
df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [29]:
#derive new column discounts, sales_price and profit
df['discount']=df['list_price'] * df['discount_percent'] * .01
df['sale_price']=df['list_price']-df['discount']
df['profit']=df['sale_price']-df['cost_price']
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0


In [31]:
#convert order_date from object data type to datetime
df['order_date']=pd.to_datetime(df['order_date'], format="%Y-%m-%d")

In [33]:
#drop cost_price, list_price and discount_percent columns from dataset
df.drop(columns = ['list_price','cost_price','discount_percent'], inplace=True)

In [15]:
#load the data into sql server
import sqlalchemy as sal

engine = sal.create_engine("mssql+pyodbc://LAPTOP-95GEC33A\\SQLEXPRESS/master?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes")
conn = engine.connect()


In [35]:
#Loading the dataset into SQL server using append 
df.to_sql('df_orders', con=conn , index=False, if_exists = 'append')

38